In [1]:
import numpy as np
from astropy.table import QTable
import astropy.coordinates as coord
import astropy.units as u

from galpy.potential import NFWPotential, HernquistPotential, MiyamotoNagaiPotential
from galpy.potential import evaluatePotentials as evalPot
from galpy.orbit import Orbit

# Importing data and transforming to Galactic Coordinates

In [2]:
# importing the data as an astropy table
# the data has been downloaded with the following criteria:
# - non null radial velocities
# - parallax_over_error > 5
# - ruwe <1 .4
# - parallax > 0.183 (to account for the parallax zero point of 0.017 mas)
gaia_rad = QTable.read("1655774299211O-result.csv")
#print(len(gaia_rad))

In [3]:
# Making a SkyCoord object to easily transform between coordinates
gaia_rad = gaia_rad[0:5]
c = coord.SkyCoord(ra = gaia_rad['ra']*u.deg,
                   dec = gaia_rad['dec']*u.deg,
                   distance = coord.Distance(parallax = (u.Quantity(gaia_rad['parallax'] ) +0.017)*u.mas),
                   pm_ra_cosdec = gaia_rad['pmra']*u.mas/u.year,
                   pm_dec = gaia_rad['pmdec']*u.mas/u.year,
                   radial_velocity = gaia_rad['radial_velocity']*u.km/u.s,
                   frame = 'icrs')


# Astropy computation

In [40]:
# transforming data to galactocentric coordinate system
galactocent = c.transform_to(coord.Galactocentric())

def Ang_Mom(coords):
    c = coords
    L_x =  (c.y * c.v_z - c.z * c.v_y)
    L_y =  (c.z * c.v_x - c.x * c.v_z)
    L_z = -(c.x * c.v_y - c.y * c.v_x) # sign is flipped by convention
    L_T = np.sqrt(L_x**2+L_y**2+L_z**2)
    return L_T, L_z # in pc * km/s 

def Energy(coords):
    c = coords
    G = 4.301 * 10**(-6) *u.kpc* (u.kilometer/u.s)**2/u.solMass
    M_h = 10**12 *u.solMass
    M_d = 9.3*10**10 *u.solMass
    M_b = 3*10**10 *u.solMass
    r_s = 21.5 *u.kpc 
    a_d = 6.5 *u.kpc  
    b_d = 0.26 *u.kpc 
    c_b = 0.7 *u.kpc
    
    HaloPot = NFWPotential(amp = G*M_h/(np.log(2)-0.5), a = r_s)
    DiskPot = MiyamotoNagaiPotential(amp = G*M_d, a = a_d, b = b_d)
    BulgePot = HernquistPotential(amp = 2*G*M_b, a = c_b)
    
    R = np.sqrt(c.x**2 + c.y**2)*u.kpc/(1000*u.pc)
    z = c.z *u.kpc/(1000*u.pc)
    Halo = evalPot(HaloPot, R, z) # in km^2/s^2
    Disk = evalPot(HaloPot, R, z) # in km^2/s^2
    Bulge = evalPot(HaloPot, R, z)# in km^2/s^2
    
    Potential = Halo + Disk + Bulge
    Kinetic = (c.v_x**2+ c.v_y**2 + c.v_z**2)/2
    
    return Kinetic , Potential # in km^2/s^2

In [41]:
a_EK , a_EP = Energy(galactocent) 
_, a_Lz = Ang_Mom(galactocent)
a_x = galactocent.x
a_y = galactocent.y
a_z = galactocent.z
a_vx = galactocent.v_x
a_vy = galactocent.v_y
a_vz = galactocent.v_z

# galpy computations

In [30]:
G = 4.301 * 10**(-6) *u.kpc* (u.kilometer/u.s)**2/u.solMass
M_h = 10**12 *u.solMass
M_d = 9.3*10**10 *u.solMass
M_b = 3*10**10 *u.solMass
r_s = 21.5 *u.kpc 
a_d = 6.5 *u.kpc  
b_d = 0.26 *u.kpc 
c_b = 0.7 *u.kpc
   
HaloPot = NFWPotential(amp = G*M_h/(np.log(2)-0.5), a = r_s)
DiskPot = MiyamotoNagaiPotential(amp = G*M_d, a = a_d, b = b_d)
BulgePot = HernquistPotential(amp = 2*G*M_b, a = c_b)
Potential = HaloPot + DiskPot + BulgePot

orbits =Orbit(c)# Orbit(c, ro = 8.2, vo = 220)

In [31]:
g_E = orbits.E(pot = Potential)
g_Lz = orbits.Lz(pot = Potential)
g_x = orbits.x()
g_y = orbits.y()
g_z = orbits.z()
g_vx = orbits.vx()
g_vy = orbits.vy()
g_vz = orbits.vz()

# Positions

In [32]:
print('x astropy:')
print(a_x)
print('x galpy:')
print(g_x)
print('y astropy:')
print(a_y)
print('y galpy:')
print(g_y)
print('z astropy:')
print(a_z)
print('z galpy:')
print(g_z)

x astropy:
[-7006.62378522 -4782.32031475 -6301.42596884 -4418.90342306
 -3946.45277294] pc
x galpy:
[6.88465241 4.66035154 6.17945537 4.2969357  3.82448576] kpc
y astropy:
[ 527.85223382 1547.86684587  851.51043883 1732.59761818 1953.5641276 ] pc
y galpy:
[0.52785223 1.54786685 0.85151044 1.73259762 1.95356413] kpc
z astropy:
[ -30.26900092  -96.64735707  -50.14436696 -123.55992275 -141.71195453] pc
z galpy:
[-0.03031255 -0.09677776 -0.05021545 -0.12370451 -0.14187499] kpc


# velocities

In [33]:
print('vx astropy:')
print(a_vx)
print('vx galpy:')
print(g_vx)
print('vy astropy:')
print(a_vy)
print('vy galpy:')
print(g_vy)
print('vz astropy:')
print(a_vz)
print('vz galpy:')
print(g_vz)

vx astropy:
[38.17883484  6.85025766 52.19156701 98.93769865 21.11563058] km / s
vx galpy:
[-36.37847901  -5.05003799 -50.39099061 -97.13774719 -19.31415305] km / s
vy astropy:
[237.51773567 254.55872067 229.45873559 220.25720422 217.09833241] km / s
vy galpy:
[224.15773567 241.19872067 216.09873559 206.89720422 203.73833241] km / s
vz astropy:
[ -1.33288865   2.15402644  -6.98136386   9.02482263 -30.06080692] km / s
vz galpy:
[ -1.86387567   1.62426266  -7.51289802   8.49146322 -30.59112767] km / s


# Energies

In [45]:
print('E astropy:')
print('K \n',a_EK, '\nP \n', a_EP, '\nTotal \n', a_EP+a_EK)
print('E galpy:')
print(g_E)

E astropy:
K 
 [28937.03739082 32425.85406415 27712.00522422 29191.67582511
 24240.6039508 ] km2 / s2 
P 
 [-2688514.91044003 -2792133.47828987 -2721993.11270562 -2807465.44962058
 -2826545.84869076] km2 / s2 
Total 
 [-2659577.87304921 -2759707.62422572 -2694281.1074814  -2778273.77379546
 -2802305.24473996] km2 / s2
E galpy:
[-930746.37410698 -974503.90571392 -946809.12832615 -984614.91049782
 -998396.99405109] km2 / s2


# Angular Momenta

In [35]:
print('Lz astropy:')
print(a_Lz)
print('Lz galpy:')
print(g_Lz)

Lz astropy:
[1684350.19943949 1227984.62786179 1490358.89937539 1144714.53472984
  898019.05436567] km pc / s
Lz galpy:
[1562.45055742 1131.88761496 1378.28094755 1057.32461173  816.92578668] km kpc / s
